In [38]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

#### Read data

In [3]:
def wrangle(filepath):
    df = pd.read_parquet(filepath)
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    return df

In [4]:
df_01 = wrangle("data/yellow_tripdata_2023-01.parquet")

#### Data description

In [7]:
print("std of the trips duration in january: ", round(df_01.duration.describe()["std"], 2))

std of the trips duration in january:  42.59


#### Drop outlier

In [8]:
# Drop outlier
new_df_01 = df_01[df_01.duration.between(1,60)]
df_01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [9]:
print("pct of the records left after dropping outlier", round(new_df_01.shape[0]/df_01.shape[0] * 100), "%")

pct of the records left after dropping outlier 98 %


#### one-hot encoding

In [33]:
categorical = ['PULocationID', 'DOLocationID']
train_data = new_df_01[categorical].astype(str)

In [34]:
train_dicts = train_data.to_dict(orient="records")

In [35]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [36]:
print("number of columns", len(dv.feature_names_))

number of columns 515


#### Split data

In [39]:
target = "duration"
y_train = new_df_01[target]

#### Train model

In [42]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

7.6492624397080675

#### evaluate model

In [43]:
def wrangle(filepath):
    df = pd.read_parquet(filepath)
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[df.duration.between(1,60)]
    return df

In [51]:
def features_selection_and_encoding(df, encoder):
    categorical = ['PULocationID', 'DOLocationID']
    features = df[categorical].astype(str)
    df_dicts = features.to_dict(orient="records")
    X = encoder.transform(df_dicts)
    target = "duration"
    y = df[target]

    return X, y

In [46]:
df_val = wrangle("data/yellow_tripdata_2023-02.parquet")
df_val.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.00,0.0,1.0,9.40,2.5,0.00,1.683333
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.00,0.0,1.0,74.65,0.0,1.25,32.083333
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.30,0.0,1.0,25.30,2.5,0.00,13.300000
5,1,2023-02-01 00:52:40,2023-02-01 01:07:18,1.0,5.10,1.0,N,148,236,1,21.9,3.50,0.5,5.35,0.0,1.0,32.25,2.5,0.00,14.633333
6,1,2023-02-01 00:12:39,2023-02-01 00:40:36,1.0,8.90,1.0,N,137,244,1,41.5,3.50,0.5,3.50,0.0,1.0,50.00,2.5,0.00,27.950000


In [52]:
X_val, y_val = features_selection_and_encoding(df_val, dv)

In [53]:
y_pred_val = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred_val)

7.81181211389241